In [3]:
import shap
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [4]:
# California Housing Prices
dataset = fetch_california_housing(as_frame = True)

X = dataset['data']
y = dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [5]:
# Prepares a default instance of the random forest regressor
model = RandomForestRegressor()

In [6]:
# Fits the model on the data
model.fit(X_train, y_train)

RandomForestRegressor()

In [7]:
# Fits the explainer
explainer = shap.Explainer(model.predict, X_test)

In [8]:
# Calculates the SHAP values - It takes some time
shap_values = explainer(X_test)

Exact explainer: 4129it [13:33,  5.03it/s]                          


In [18]:
shap_values.values()

AttributeError: 'Exact' object has no attribute 'values'